<img src="images/Picture0.png" width=200x />

# Notebook 04 - Adding Multiple Variables and Constraints

### Covered in this notebook:

* Using addVars method
* Using addConstrs method

To wrap up our introduction of the fundamentals, let's explore ways to quickly add many variables and constraints.  Some professional Gurobi models involve thousands of variables and tens of thousands of constraints: how do they do this practically?  Let's look into it.

## Multiple variables

### Option 1: Add variables using a loop

Of course, we can use a loop structure to add many decision variables to a model.  We will not assign these decision variables to Python variables (`x = model.addVar() for i in range(10)` would <i>not</i> be useful code, because we would reassign the variable over and over again).  But we need some assignment to let us distinguish between our decision variables and retrieve them when we need them.  This is where we use the optional `name` argument of `addVar`, and the method `Model.getVarByName()` (or, analogously, `Model.getConstrByName()`) to retrieve it.  The `name` argument instantiates the `VarName` attribute of the decision variable.

In [1]:
import gurobipy as gp
from gurobipy import GRB

In [2]:
# create a practice model
n = gp.Model()

# add four continuous variables with names x0, x1, x2, x3
for i in range(4):
    n.addVar(name = 'x' + str(i))

Set parameter Username


### Option 2: Use the addVars method

Alternatively, we can use Gurobi's built-in methods for adding multiple variables at once.  This is often the quicker way forward, since loops take time to iterate, whereas this method instantiates all of the variables designated at the same time.

The `Model.addVars()` method creates a tupledict object containing the newly created decision variables.  The keys for the decision variables come from their indices, which are <i>very</i> flexible.  `addVars` takes `*indices` as a wildcard argument; that is, any number of arguments can be passed in.  The type of those arguments can be broad as well.  `*indices` can take an integer, multiple integers, a constant, a tuple, a list, a combination of the previous… there are many possibilities, not all of which are relevant to us as we are learning, but which are interesting in their own right.  Gurobi then creates a multi-dimensional tupledict of variables according to these indices.  [{SHOULD I WRITE ABOUT HOW TO INDEX A VARIABLE AS WELL??}]()

`addVars` also takes the arguments we are familiar with from `addVar`; namely, `lb`, `ub`, `obj`, `vtype`, and `name`.

Create a model to practice with, then add variables to it using the appropriate means.

In [ ]:
# examples: ask the students about the indices they should have, then ask
## them about the shape you want to create and let them make their own indices

In [35]:
# add 5 variables, and name the tupledict "variables"
variables = n.addVars(5, name="variables")

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>}

In [7]:
# add three two-dimensional variables
n.addVars(3,1)

{(0, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 0): <gurobi.Var *Awaiting Model Update*>}

In [8]:
# add variables with indices 'a', 'b', and 3
n.addVars(('a','b',3))

{'a': <gurobi.Var *Awaiting Model Update*>,
 'b': <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>}

In [9]:
# add nine three-dimensional variables
n.addVars(3,3,1)

{(0, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (0, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (1, 2, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 0, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 1, 0): <gurobi.Var *Awaiting Model Update*>,
 (2, 2, 0): <gurobi.Var *Awaiting Model Update*>}

### Adding multiple constraints

When it comes to adding multiple constraints at once, the `Model.addConstrs()` method is our friend.  We won't discuss adding constraints in a loop because the method `addConstrs` actually works according to that idea already.  `addConstrs` takes a `generator` expression which defines a constraint over a loop.  (The other argument it takes is `name`.)

This is best understood through examples, so we'll jump in quickly.  Before we do, let's take a moment to note a useful built-in method in Gurobipy: `tupledict.sum()`, which sums over a tupledict, e.g. a tupledict of variables.  This lets us create linear expressions across variable dictionaries.  `tupledict.sum()` takes the argument of a `pattern`, where the index values to be included in the sum are given for each position.  A '`*`' value indicates that all values are to be included for that position.

Look at the construction of the two examples below, then build your own constraints according to the prompts.

In [41]:
m1 = gp.Model()
x = m1.addVars(4)
m1.update()

# constraint: each of 4 variables less than or equal to 1
m1.addConstrs(x[i] <= 1 for i in x)

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [43]:
m2 = gp.Model()
x = m2.addVars(4)
m2.update()

# constraint: the total sum of all variables must be less than 50
m2.addConstr(x.sum('*') <= 50)

<gurobi.Constr *Awaiting Model Update*>

In [44]:
m3 = gp.Model()
x = m3.addVars(4, 4)
m3.update()

# constraint: the sum of all variables sharing a first index must each be less than 50
m3.addConstrs(x.sum(i, '*') <= 50 for i in range(4))

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>}

In [45]:
m4 = gp.Model()
x = m4.addVars(4)
m4.update()

# constraint: the sum of any two variables drawn randomly with replacement must be less than 1
m4.addConstrs(x[i] + x[j] <= 1 for i in x for j in x)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>}

In [10]:
m5 = gp.Model()
x = m5.addVars(4)
m5.update()

# constraint: the sum of any two variables drawn randomly with replacement must be less than 1,
# unless the variable is added to itself
m5.addConstrs(x[i] + x[j] <= 1 for i in x for j in x if i != j)

{(0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>}

In [46]:
m5 = gp.Model()
x = m5.addVars(4)
y = m5.addVars(5)
m5.update()

# constraint: any given variable in x is less than any given variable in y
m5.addConstrs(x[i] <= y[j] for i in x for j in y)

{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,
 (3, 2): <gurobi.Constr *Awaiting Model Update*>,
 (3, 3): <gurobi.Constr *Awaiting Model Update*>,
 (3, 4): <gurobi.Constr *Awaiting Model Update*>}

## Exercise: Growing Your Landscaping Business

Now it's your turn again.  You're rethinking your landscaping business from Notebook 02 and deciding on what types of greenery to focus on in the next quarter.

The nursery has offered you a variety of species of shrub and tree to choose from.  The amount that each plant increases your revenue varies, as does the time they take to plant.  You'd like to know which plants to focus on to create your optimal job.

The `shrubs.csv` and `trees.csv` files give the names of the plants available to you, the amount that each raises your revenue, and the average time each takes to plant.

The rest of your conditions are unchanged:

* Your patrons still prefer at least a three-to-one ratio of shrubs to trees.
* You still don't want to use more than 40 hours on greenery for a given job.
* Your deal with the arbor still allows you to take out 100 credits' worth of greenery per job.  A shrub still costs 1 credit and a tree still costs 5 credits.
* Your revenue still starts at $5000.  (However, it increases by a variable amount per tree or shrub.)

Which plants give you your optimal job, and how much do you earn?

<i>Note: If it's helpful, recall that the Gurobi model automatically defines an objective function based on the objective weights `obj` of the decision variables, with a default sense of minimization.</i>

In [48]:
import csv

sName = []
sRevenue = []
sPlant = []

with open('data/N02_data/ex2_shrubs.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        sName.append(row[0])
        sRevenue.append(float(row[1]))
        sPlant.append(float(row[2]))

tName = []
tRevenue = []
tPlant = []

with open('data/N02_data/ex2_trees.csv', mode='r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        tName.append(row[0])
        tRevenue.append(float(row[1]))
        tPlant.append(float(row[2]))

In [49]:
# instantiate model
p = gp.Model()

In [50]:
# add variables
import numpy as np

shrubs = p.addVars(len(sName), vtype=GRB.INTEGER, name=sName, obj=np.array(sRevenue)*-1)
trees = p.addVars(len(tName), vtype=GRB.INTEGER, name=tName, obj=np.array(tRevenue)*-1)

In [51]:
# add constraints
# at least 3 shrubs per tree:
p.addConstr(shrubs.sum() >= 3 * trees.sum())

# less than 40 hours per job:
p.addConstrs(shrubs[i] * sPlant[i] + trees[j] * tPlant[j] <= 40 for i in shrubs for j in trees)

# 100 credits worth of greenery per job:
p.addConstr(shrubs.sum() + 5 * trees.sum() <= 100, name="credits")

<gurobi.Constr *Awaiting Model Update*>

In [52]:
p.optimize()

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 23.5.0 23F79)

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 342 rows, 37 columns and 754 nonzeros
Model fingerprint: 0x768b0f97
Variable types: 0 continuous, 37 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 5e+00]
  Objective range  [3e+01, 5e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Found heuristic solution: objective -3472.000000
Presolve removed 143 rows and 0 columns
Presolve time: 0.00s
Presolved: 199 rows, 37 columns, 468 nonzeros
Variable types: 0 continuous, 37 integer (0 binary)
Found heuristic solution: objective -5159.000000

Root relaxation: objective -8.500000e+03, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 -8500

<strong>What is your optimal job?</strong>

Print your nonzero plant variables and their values, and print your overall revenue.

In [54]:
for v in p.getVars():
    if v.x != 0:
        print(v.varName, '=', v.x)

print('revenue:', 5000-p.objVal)

fuchsia = 40.0
Ageratina altissima = 12.0
revenue: 13360.0
